## Multiprocesamiento - Aumentando la capacidad de computo

Hay muchas tecnicas para aumentar la capacidad de computo de un CPU, pero siempre hay un limite. Si miramos para atras en la historia, los procesadores tenian algunos megahertz si nos vamos a principios de los 90 hablamos de decenas de megahertz, si vamos a fines de 90 hablamos de centeneas de megahertz, ya en los 2000 llegamos a los gigahertz y ahora seguimos en los gigahertz, osea que no se ha multiplicado como venia antes, el motivo son limitaciones fisicas, en la decada del 80 habia limitaciones tecnologicas, no habia tecnologia para achicar las cosas. Ahora el problema es un limite fisico, los transistores, al la materia ser granular ya no se comporta como un continuo y el hecho de que hayan atomos discretos hace que no lo pueda achicar mas. Entonces, cuando queremos crecer por encima de lo que hoy es un limite, no queda mas alternativa que utilizar multiples CPUs, aparecen varios problemas. El principal problema es como dividir la tarea. Para tareas con poca interaccion utilizar n cpus resultara en n veces mas velocidad. Pero para sub-tareas con mucha interaccion mutua, se debe compartir memoria y el proceso se hace menos optimo y mas complicado.

## Escenarios

Cuando hablamos de sistemas multiprocesadores tenemos dos esquemas bien divididos, los de memoria compartida o shared-memory, son tambien los que mas acostumbrados estamos. En los sistemas de memoria compartida, todos acceden a la memoria en las mismas condiciones, nadie tiene prioridad o privilegio para acceder a la memoria. Hay un tema importante que es el sincronismo entre las CPUs. En la medida que logremos separacion eso es un beneficio. Porque entran los sistemas mas comunes como las notebooks en esto? dijimos que esto funcionaba bien cuando no compartiamos la memoria y dijimos que los procesos tienen todos espacios de memoria distintos, solamente cuando expresamente se le pide al SO un area de memoria compartida que es un caso muy puntual de muy pocos procesos, se da ese caso de poder pelear por un recurso. En circunstancias normales uno corre multiples tareas y esas tareas hacen cosas distintas y ahi es donde logramos no interrelacion entre tareas, entonces tener el doble de cores implicara el doble de velocidad. Por otro lado tenemos los sistemas de snared interconnect en donde tenemos dos tipos de memoria, cada CPU tiene su propia memoria en donde trabaja, ejecuta los programa y calcula los resultados, pero se dispone de una segunda memoria de interconexion compartida en la cual se hace el intercambio de los datos de una CPU y otra. En que difieren princiaplmente es que estos son mas dificiles de construir en cuanto al hardware porque hay que resolver el problema de la contencion por el recurso pero la realidad es que son faciles de programar porque cada programa asume que tiene todo, los otros, son mas dificiles de construir porque no solamente hay que resolver la interaccion entre CPU y memoria sino que tengo que resolver la interconeccion de esas posiciones de memoria, el programador tiene que saber que hay abajo para poder dividir su problema en distintas tareas de forma que tenga un conjunto finito de datos de entrada, de resultados y como van a interactuar los procesos. Entonces, se complica mas la programacion porque el programador no trabaja de forma transparente de lo que hay abajo.

## Shared Memory - Multiprocessors / UMA (Uniform Memory Access)

Cuando hablamos del caso de memoria compartida entonceas existe una unica zona de memoria que ven todos los CPUs, en el segundo caso solo un pedacito de memoria era el compartido y habia un monton de memoria individuales de las CPUs que las otras no podian acceder. Vamos a analizar el caso en que todas la CPUs ven la totalidad del espacio de memoria, podemos dividirlo en dos categorias, una es UMA: acceso uniforme, todos tinen el mismo privilegio en todas las zonas de memoria, es decir no solamente ven todo sino que ven todo en igualdad de condiciones, a diferencia de las NUMA, todos ven todo pero es como que hay preferencia o mas velocidad en un sector. Las que son UMA escencialmente hay tres arquitecturas que se usan, estamos hablando de maquinas que tienen miles de cores como las supercomputadoras. Las primeras que son las que tenemos en las notebooks son las de single bus, estas tienen un solo bus de direcciones y un solo bus de datos, y cada core cuando quiere acceder le habla a un MMU y esa unidad de manejo de memoria coordina los accesos de los distintos cores para acceder a una unica memoria, son mas faciles de hacer pero obviamente es valido y funcional para no mas de 10 o 20 CPUs, es decir no hablamos de mas de 10 o 20 CPUs, llega un momento que el bus comienza a ser un cuello de botella porque por ese bus pasan todas las transacciones de todas las CPUs, si empiezo a poner muchos cores empiezo a tener un cuello de botella y pierdo performance. Para evitar eso es que se colocan caches entre cada uno de los CPUs y la memoria para evitar que todos los accesos de la CPU terminen en acceso a memoria porque sino claramente el cuello de botella aparece antes. Si no tengo ningun cache y cada acceso a la CPU tiene que ir a memoria entonces claramente todos tienen que ir a memoria y el cuello de botella ya esta ahi. Hay que tener un protocolo de coherencia de cache en donde si alguien modifica sobre el cache, todo lo que esta ya sea al costado o abajo sepa que hasta que ese dato no llegue no lo voy a poder usar. Vamos al otro extremo, hasta aca teniamos un unico bus y n CPUs, que pasa si hacemos una matriz, entonces una CPU podria acceder por distintos caminos a cualquiera de las memorias, la memoria esta partida en pedazos pero esto reduce mucho la cantidad de choques, esto es a expensas de mucho hardware, estamos poniendo caminos reduantes para todas las memorias, es mas facil porque elimine los caches entonces no hay contenciones, no hay que andar usando un protocolo de coherencia de cache, pero obviamente esto crece con el cuadrado de pares memoria-CPU. Hay una solucion de compromiso que es lo que se llama red omega y escencialmente lo que haces es que cada uno de los nodos toma dos entrada y genera dos salidas, de esa forma, si bien es verdad que cada CPU no tiene un camino propio a la memoria, pero es una opcion de compromiso que se utiliza hoy en dia en las supercomputadoras, lo que pasa es que la cantidad de nodos necesarios para la interconexion en vez de crecer con el cuadrado de pares memoria-CPU crece con el logaritmo, entonces obviamnete la cantidad de hardware involucrado es mas chico. La mayor parte de la programacion de estos sistemas involucra conocer por abajo el sistema para saber como distribuyo las estructuras en las distintas memorias, etc. No es que el SO lo haga todo por mi y resuelva todo.

## Shared Memory - Multiprocessors / NUMA (Non Uniform Memory Access)

Acceso de memoria no uniforme, tienen un espacio de memoria unico pero ahora cada una de las CPUs tiene una memoria de uso no exclusivo pero de mas rapido acceso que los demas. Lo que es importante para que se mantenga la categoria de memoria compartida es que todas las CPUs vean la totalidad del espacio de memoria, en las UMA cualquier posicion tarda lo mismo en las NUMA no algunas posiciones son mas rapidas y otras mas lentas pero se puede ver todo el espacio de memoria. En estos casos, se puede o no usar el cache, dependera de la arquitectura en particular y sobretodo de cual es esa diferencia de velocidad entre la memoria local y la memoria remota.

## Shared Interconnect - Multicomputers

Este es el caso en el que las CPUs no tienen el espacio de memoria compartido en su totalidad, solamente tienen alguna zona de memoria comun donde se intercambian datos. Para que esto funcione tenemos que tener dos condiciones, primero una forma razonablemnte rapida de intercambiar los datos entre los distintos bloques, osea como no vemos la misma posicion de memoria entonces tengo que de alguna forma usar esa interconeccion para notificar a los otros nodos que tengo un resultado que tal posicion de memoria cambio. Por otro lado, para estos es imprescindible que las aplicaciones conozcan el hardware. Los SOs lo maximo que pueden hacer en estos casos es encargarse de repartir el mensaje a los distintos nodos, una alternativa es usar el DMA, el DMA era un segundo modulo que competia con el procesdaor a los efectos de acceder a memoria. El DMA es una arquitectura que es valida en un hardware relativamnte pequenio. Hay SOs que trabajan con distribucion de tareas tipo red, dado un mensaje y un destinatario, relevo en el SO entregarle ese mensaje al otro nodo. Otros trabajan con una cola de mensajes entrantes donde recibo las notificaciones de las cosas que los otros nodos calcularon y me quieren pasar a mi. Hay un formato que se llama RPC que es un formato estandar para llamar procedimientos que no estan en la misma maquina. Hay compiladores que se pueden usar donde uno define un conjunto de procedimietos y uno los puede llamar desde su codigo. Como si estuviesen en una biblioteca. En realidad, en vez de llamarse a la biblioteca, el codigo ese lo que va a hacer es agarrar los parametros formatearlos de determinada forma, mandarlos por la red al otro lado y ese procedimiento que fisicamnete se ejecute en otro lugar, esos resultados van a volver y van a ser procesados. Hoy en dia algo muy similar son los webservices. Hoy en dia la mayor parte de las funciones que hay entre empresas se hace a traves de webservices. Estos servicios se prestan normalmente desde datacenters. La memoria distribuida es un consepto dificil de precisar, facil de entender, tengo una memoria distribuida por toda la red cuyos pedasos los tienen cada uno, admite capacidades enormes pero hay un tema de velocidad. Estas son todas alternativas donde de alguna forma se excede la cantidad de recursos que uno puede tener en un solo nodo.

## Sistemas distribuidos

A veces podemos llegar a tener directamente sistemas distribuidos, estos sistemas estan directamente separados geograficamente unos del otro, obviamnete tenemos que lograr la minima interaccion entre los nodos. Los sistemas distribuidos no solamente se usan para ganar en perfomance sino que tambien para ganar en confiabilidad, estamos hablando de que es muy poco probable de que si hacemos el mismo calculo en dos maquinas en paralelo que una se rompa y que no obtengamos el resultado pero hay aplicaciones donde inclusive esa circunstancia tan rara es inaceptable. Estos sistemas distribuidos son como una tercera categoria, no comparten ninguna parte de la memoria, tienen sistemas independientes que trabajan por separado y a lo sumo se intercambian algun bloque de informacion. Por ejemplo uno calcula la velocidad entonces cuando alguien quiera la velocidad se la va a pedir a el, asi con otros parametros. Obviamnete son nodos que tienen una funcion especifica, un ejemplo es un avion. Hay una unidad que se encarga de detectar en base a ciertos sensores que tiene cual es la velocidad y la altitud del avion, esa es una unidad. Hay otra unidad que se encarga de encontrar cual es la posicion, y esa informacion se comparte a traves de mensajes, hay otras computadoras por ejemplo la que controla el curso obviamente tiene que pedirle a la que calcula la posicion donde esta y a la que calcula la velocidad para poder hacer cambios y actuar sonbre por ejemplo en un piloto automatico. Y son todas computadoras independientes. Los displays que ven los pilotos son otra computadora que toma los mismos datos que estan tomando las otras computadoras y la saca para mostrarsela al piloto. Todas estas cosas trabajan independientes, los motores de los aviones tambien tienen computadoras propias. Estas cosas no solamente se usan para aumentar la cantidad de recursos sino que tambien son muy utlizadas para aplicaciones de mucha confiabilidad. Tenemos multiples sistemas haciendo lo mismo y quee esos multiples sistemas sean comparados por otro para asegurarse que nada difiere. Estos casos practicamente no tienen un SO, hay poco interes de que alguine ajeno quiera generar codigo, en conclusion estos sistemas distribuidos son muy dedicados para esa aplicacion y no para otra. Dento de estos sistemas distribuidos podemos hacer algunas clasificaciones para encuadrar las distintas cosas que hoy en dia hay en el mercado, los primeros son los cliente servidor, tengo distintos servidores que calculan distintas cosas y tengo clientes que en funcion de que necesitan tienen que saber a quien esta preguntado. En realidad la informacion es casi unidireccional. Hay otros que directamente son totalmente anarquicos, cualquiera puede requerir informacion de cualquiera y se estan pasando informacion entre ellos permanentemente. Hay algunas alternativas intermedias, los que se llaman middleware, en donde hay un pequenio SO que se encarga de compartir los datos. Despues hay algunos que dividen en mas capas, no solamente en capa de SO y de tarea sino que divide las capas por ejemplo en interfaz con el usuario, procesamiento y datos. Esto es muy comun en la industria de servicios, por ejemplo un banco.

## Maquinas virtuales

Hay sistemas de maquinas virtuales que soportan la utilizacion de nodos NUMA. Los nodos NUMA en maquinas virtuales permiten que se ejecuten partes en distintos lugares. Queda claro, que los cores de mi maquina van a ir mucho mas rapido a mi RAM que si trataran de ir a la RAM de un core que esta corriendo en la maquina de al lado, tengo entonces RAM mas rapida y RAM mas lenta, sin embargo se permite que todos vean el mismo espacio de memoria. El Hyper-V viene con el windows y es una alternativa que permite repartir maquinas mas grandes. VMWare tambien lo soporta. Divide los recursos disponibles en nodos NUMA, que luego pueden asignarse a distintas maquinas. 